In [ ]:
import numpy as np
import pandas as pd 
from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))
import datetime
import time

In [ ]:
train1 = pd.read_csv("../input/train.csv", nrows=100000)

In [ ]:
from sklearn import svm

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
test1 = pd.read_csv("../input/test.csv", nrows=100)

In [ ]:
train1["srch_ci"] = pd.to_datetime(train1["srch_ci"], format='%Y-%m-%d', errors="coerce")

In [ ]:
test1["srch_ci"] = pd.to_datetime(test1["srch_ci"], format='%Y-%m-%d', errors="coerce")

In [ ]:
train1["srch_co"] = pd.to_datetime(train1["srch_co"], format='%Y-%m-%d', errors="coerce")

In [ ]:
test1["srch_co"] = pd.to_datetime(test1["srch_co"], format='%Y-%m-%d', errors="coerce")

In [ ]:
train1["stay_span"] = (train1["srch_co"] - train1["srch_ci"]).astype('timedelta64[D]')

In [ ]:
test1["stay_span"] = (test1["srch_co"] - test1["srch_ci"]).astype('timedelta64[D]')

In [ ]:
train2 = pd.to_datetime(train1["date_time"])

In [ ]:
test2 = pd.to_datetime(test1["date_time"])

In [ ]:
train2 = pd.DataFrame(train2)
test2 = pd.DataFrame(test2)

In [ ]:
train1['year'] = train2['date_time'].dt.year
train1['month'] = train2['date_time'].dt.month
train1['day_of_week'] = train2['date_time'].dt.dayofweek
train1['day'] = train2['date_time'].dt.day
train1['hour'] = train2['date_time'].dt.hour

In [ ]:
test1['year'] = test2['date_time'].dt.year
test1['month'] = test2['date_time'].dt.month
test1['day_of_week'] = test2['date_time'].dt.dayofweek
test1['day'] = test2['date_time'].dt.day
test1['hour'] = test2['date_time'].dt.hour

In [ ]:
train1["date_time"] = pd.to_datetime(train1["date_time"], format='%Y-%m-%d', errors="coerce")
train1["srch_ci"] = pd.to_datetime(train1["srch_ci"], format='%Y-%m-%d', errors="coerce")
train1["search_span"] = (train1["srch_ci"] - train1["date_time"]).astype('timedelta64[D]')

In [ ]:
test1["date_time"] = pd.to_datetime(test1["date_time"], format='%Y-%m-%d', errors="coerce")
test1["srch_ci"] = pd.to_datetime(test1["srch_ci"], format='%Y-%m-%d', errors="coerce")
test1["search_span"] = (test1["srch_ci"] - test1["date_time"]).astype('timedelta64[D]')

In [ ]:
train1.ix[(train1['hour'] >= 10) & (train1['hour'] < 18), 'hour'] = 1
train1.ix[(train1['hour'] >= 18) & (train1['hour'] < 22), 'hour'] = 2
train1.ix[(train1['hour'] >= 22) & (train1['hour'] == 24), 'hour'] = 3
train1.ix[(train1['hour'] >= 1) & (train1['hour'] < 10), 'hour'] = 3

In [ ]:
test1.ix[(test1['hour'] >= 10) & (test1['hour'] < 18), 'hour'] = 1
test1.ix[(test1['hour'] >= 18) & (test1['hour'] < 22), 'hour'] = 2
test1.ix[(test1['hour'] >= 22) & (test1['hour'] == 24), 'hour'] = 3
test1.ix[(test1['hour'] >= 1) & (test1['hour'] < 10), 'hour'] = 3

In [ ]:
train1 = train1.drop('srch_ci', axis=1)

In [ ]:
test1 = test1.drop('srch_ci', axis=1)

In [ ]:
train1 = train1.drop('srch_co', axis=1)

In [ ]:
test1 = test1.drop('srch_co', axis=1)

In [ ]:
train1 = train1.drop('date_time', axis=1)

In [ ]:
test1 = test1.drop('date_time', axis=1)

In [ ]:
desti1 = pd.read_csv("../input/destinations.csv")

In [ ]:
Xt = desti1.ix[:,1:150]

In [ ]:
yt = desti1.ix[:,0:1]

In [ ]:
pca1 = PCA(n_components=3)
X_train = pca1.fit(Xt).transform(Xt)

In [ ]:
X_train1 = pd.DataFrame(X_train)

In [ ]:
X_train1["srch_destination_id"] = desti1["srch_destination_id"]

In [ ]:
X_train1.info()

In [ ]:
pca2 = PCA(n_components=4)
X_test = pca2.fit(Xt).transform(Xt)

In [ ]:
X_test1 = pd.DataFrame(X_test)

In [ ]:
X_test1["srch_destination_id"] = desti1["srch_destination_id"]

In [ ]:
X_train1.columns = ['d0','d1','d2','srch_destination_id']

In [ ]:

X_test1.columns = ['d0','d1','d3','d4','srch_destination_id']

In [ ]:
train1 = train1.join(X_train1, on = 'srch_destination_id', how = 'left', rsuffix='dest')

In [ ]:
test1 = test1.join(X_test1, on = 'srch_destination_id', how = 'left', rsuffix='dest')

In [ ]:
train1 = train1.drop("srch_destination_iddest", axis=1)

In [ ]:
test1 = test1.drop("srch_destination_iddest", axis=1)

In [ ]:
train1.fillna(-1, inplace=True)

In [ ]:
test1.fillna(-1, inplace=True)

In [ ]:
hotelCluster = train1.ix[:,'hotel_cluster']

In [ ]:
hotelCluster1 = pd.DataFrame(hotelCluster)

In [ ]:
hotelCluster.shape

In [ ]:
train1 = train1.drop('hotel_cluster', axis=1)

hc = np.array(hotelCluster1)

hc.shape

In [ ]:
print(hotelCluster.shape)

In [ ]:
clf = svm.SVC()

In [ ]:
clf.fit(train1, hotelCluster)